In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from langchain.docstore.document import Document


2025-06-12 00:48:28.726169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749689309.113675      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749689309.229095      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from dotenv import load_dotenv

load_dotenv()

False

In [4]:
import pandas as pd

books=pd.read_csv("/kaggle/input/books-cleaned-csv/books_cleaned.csv")

In [5]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_sub_title,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [6]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [7]:
books["tagged_description"].to_csv("tagged_description.txt",sep="\n",
                                   index=False,
                                   header=False
                                   )

In [8]:
raw_documents=TextLoader("/kaggle/working/tagged_description.txt",encoding="utf-8").load()
text_splitter=CharacterTextSplitter(chunk_size=0,chunk_overlap=0,separator="\n")
documents=text_splitter.split_documents(raw_documents)

In [9]:
documents[0]

Document(metadata={'source': '/kaggle/working/tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of li

In [10]:
torch.cuda.is_available()
device=('cuda'if torch.cuda.is_available() else "cpu")

In [11]:
class SentenceTransformerEmbedding:
    def __init__(self, model_name, device='cpu'):
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts):
        return self.model.encode(texts, batch_size=32, show_progress_bar=True).tolist()

    def embed_query(self, text):
        return self.model.encode([text], show_progress_bar=False).tolist()[0]

embedding_model = SentenceTransformerEmbedding('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

texts = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]

db_books = Chroma.from_texts(
    texts=texts,
    embedding=embedding_model,
    metadatas=metadatas,
    persist_directory="./chroma_db"
)

db_books.persist()

print(f"Number of documents in DB: {db_books._collection.count()}")

# Now the embedding_model has embed_query, so similarity_search should work
results = db_books.similarity_search("story about A young wizard attends a magical school in England.", k=3)

for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("-----")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/163 [00:00<?, ?it/s]

Number of documents in DB: 5197
9780590254762 Four English school children enter the magic land of Narnia through the back of a wardrobe and assist Aslan, the golden lion, in defeating the White Witch who has cursed the land with eternal winter.
{'source': '/kaggle/working/tagged_description.txt'}
-----
9780060845247 Four English school children enter the magic land of Narnia through the back of a wardrobe and assist Aslan, the golden lion, in defeating the White Witch who has cursed the land with eternal winter.
{'source': '/kaggle/working/tagged_description.txt'}
-----
9780439411875 Failing out of magic school, Rhianna meets a wizard who realizes she is a Wild Talent, possessing a difficult-to-manage power, and when a dragon comes to town, it's up to Rhianna to master her craft or face certain doom.
{'source': '/kaggle/working/tagged_description.txt'}
-----


/tmp/ipykernel_35/252810268.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db_books.persist()


In [12]:
import shutil

# Move the entire chroma_db folder to /kaggle/working
shutil.move('./chroma_db', '/kaggle/working/chroma_db')


In [13]:
shutil.make_archive('/kaggle/working/chroma_db', 'zip', '/kaggle/working/chroma_db')


'/kaggle/working/chroma_db.zip'

In [14]:
books[books["isbn13"]==int(documents[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_sub_title,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...


In [15]:
def retrieve_semantic_recommendation(query:str , top_k: int=10)->pd.DataFrame:
    recommds=db_books.similarity_search(query,k=50)
    books_list=[]
    for i in range(0,len(recommds)):
        books_list+=[int(recommds[i].page_content.strip('""').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [16]:
retrieve_semantic_recommendation("A story about a magical world and adventures of a young wizard")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_sub_title,tagged_description
102,9780060177249,0060177241,The thief of always,Clive Barker,Fiction,http://books.google.com/books/content?id=jKsrA...,After a mysterious stranger promises to end hi...,1992.0,4.19,225.0,22123.0,The thief of always: a fable,9780060177249 After a mysterious stranger prom...
146,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,"The Lion, the Witch and the Wardrobe (picture ...",9780060556501 Narnia: A magical land full of w...
205,9780060745905,0060745908,Wicked Musical Tie-in Edition,Gregory Maguire,Fiction,http://books.google.com/books/content?id=MLEuj...,When Dorothy triumphed over the Wicked Witch o...,2004.0,3.53,409.0,2747.0,Wicked Musical Tie-in Edition: The Life and Ti...,9780060745905 When Dorothy triumphed over the ...
206,9780060747220,0060747226,Son of a Witch,Gregory Maguire,Fiction,http://books.google.com/books/content?id=5-3f3...,"Ten years after the publication of Wicked, bel...",2006.0,3.47,352.0,51493.0,Son of a Witch: A Novel,9780060747220 Ten years after the publication ...
220,9780060765453,0060765453,The Chronicles of Narnia Movie Tie-in Edition ...,C. S. Lewis,Fiction,http://books.google.com/books/content?id=4qVxv...,When Digory and Polly try to return the wicked...,2005.0,4.26,766.0,1099.0,The Chronicles of Narnia Movie Tie-in Edition ...,9780060765453 When Digory and Polly try to ret...
256,9780060845247,0060845244,"The Lion, the Witch and the Wardrobe Read-Alou...",C. S. Lewis;Pauline Baynes,Juvenile Fiction,http://books.google.com/books/content?id=deiyG...,Four English school children enter the magic l...,2005.0,4.21,189.0,56.0,"The Lion, the Witch and the Wardrobe Read-Alou...",9780060845247 Four English school children ent...
423,9780064410342,006441034X,Howl's Moving Castle,Diana Wynne Jones,Juvenile Fiction,http://books.google.com/books/content?id=f3RHP...,"In the land of Ingary, such things as spells, ...",2001.0,4.30,329.0,148957.0,Howl's Moving Castle,"9780064410342 In the land of Ingary, such thin..."
646,9780140367430,0140367438,The Enchanted Castle,E. Nesbit,Juvenile Fiction,http://books.google.com/books/content?id=75V6x...,"E. Nesbit's classic story of how Gerald, Cathy...",2004.0,3.85,291.0,6124.0,The Enchanted Castle,9780140367430 E. Nesbit's classic story of how...
771,9780141303352,0141303352,Mr Majeika and the School Trip,Humphrey Carpenter,Juvenile Fiction,http://books.google.com/books/content?id=YJ8eq...,"More amazing adventures with Mr Majeika, the e...",1999.0,3.81,89.0,45.0,Mr Majeika and the School Trip,9780141303352 More amazing adventures with Mr ...
778,9780141311401,0141311401,Boy,Roald Dahl,"Authors, English",http://books.google.com/books/content?id=GmE-P...,'Throughout my young days at school and just a...,2001.0,4.10,176.0,42784.0,Boy: Tales of Childhood,9780141311401 'Throughout my young days at sch...
